# Lesson 3 - Euler McLaurin and Bernoulli numbers

## Learning Outcomes:

The student will learn to manipulate symbolic expressions: 
    - differentiate
    - simplify
    - substitute (evaluate)
They will also learn following concepts from SageMath:
- the `_` variable
- taylor expansions
- coefficients


## Mathematical problem:

One of the methods available for computing the Riemann zeta function is the Euler - McLaurin summation formula, which in general says that if $f(x)$ is infinitely differentiable on $\mathbb{R}$, $a \le b$ are integers and $k>0$ then:
$$
\sum_{n=a}^{b} f(n) = \int_{a}^{b} f(x) dx  + \frac{f(a)+f(b)}{2} + 
\sum_{j=1}^{k}\frac{B_{j}}{(j)!} \left( f^{(j-1)}(b) - f^{(j-1)}(a)\right)
+ \frac{(-1)^{k-1}}{(k)!} \int_{a}^{b} B_{k}(x-[x])f^{(k)}(x) dx.
$$
Here the constants $B_{k}$ are Bernoulli numbers, $B_j(x)$ Bernoulli plynomials and $[x]$ is the integer part of $x$.

### Bernoulli numbers
The Bernoulli numbers are a sequence of rational numbers that appear in many parts of number theory and are in particular closely related to the Riemann zeta function 
$$\zeta(s)= \sum_{n=1}^{\infty} n^{-s}$$
For instance 
$$
\zeta(2)=\frac{\pi^2}{6}, \quad \zeta(4)=\frac{\pi^4}{90}\ldots
$$
and in general 
$$
\zeta(2k)=(-1)^{k-1}\frac{(2\pi)^{2k}}{2(2k)!}B_{2k}
$$
The Bernoulli numbers $B_k$ can be defined in terms of a generating series:
$$
F(x) = \frac{x}{e^x-1} = \sum_{k=0}^{\infty} B_k \frac{1}{k!}x^k
$$
and we will see how we can use symbolic expressions in SageMath to find their values (this is by far not the most efficient way...)

## Symbolic expressions in SageMath
When starting Sage the name 'x' is defined as a symbolic variable and other variables have to be declared using
the `var(x,y,z,...)` command.
Symbolic expressions can treated in various ways:
- differentiation
- simplifications


In [2]:
x

x

In [4]:
y

NameError: name 'y' is not defined

In [5]:
var('x','y','z')

(x, y, z)

In [6]:
y*z

y*z

### Differentiation

In [8]:
g = 1/(x**2+y**2)
print(f"g\t= {g}")
print(f"dg/dx\t= {diff(g,x)}")
print(f"d^2g/dxdy\t= {diff(g,x,y)}")

g	= 1/(x^2 + y^2)
dg/dx	= -2*x/(x^2 + y^2)^2
d^2g/dxdy	= 8*x*y/(x^2 + y^2)^3


### Simplification

In [11]:
f = x*y/(x**2 + y**2 ); f
z=diff(f,x,y);z

8*x^2*y^2/(x^2 + y^2)^3 - 2*x^2/(x^2 + y^2)^2 - 2*y^2/(x^2 + y^2)^2 + 1/(x^2 + y^2)

In [12]:
z.simplify_full()


-(x^4 - 6*x^2*y^2 + y^4)/(x^6 + 3*x^4*y^2 + 3*x^2*y^4 + y^6)

We can now try to find the Bernoulli numbers:

In [58]:
F = x/(e**x -1); F # The generating function

x/(e^x - 1)

Try to find the first Taylor coefficient:

In [23]:
g=derivative(F,x,1); g # another name for .diff()

-x*e^x/(e^x - 1)^2 + 1/(e^x - 1)

In [24]:
g.simplify_full() # yes - there are other types of simplifications.
g.substitute(x=0)

ValueError: power::eval(): division by zero

We cant' just divide by 0 / 0. We need L'hopitals rule!

In [25]:
g.numerator().diff(x) / g.denominator().diff(x)

-1/2*x/(e^x - 1)

Still of the form 0 /0. Need one more derivative!

In [30]:
# Note that the second parameter gives the number of times we differentiate
g.numerator().diff(x,2) / g.denominator().diff(x,2)  

_.simplify_full().substitute(x=0)  # _ is the last evaluated expression

-1/2

So the first Bernoulli number $B_1=-\frac{1}{2}$.
This method is a bit cumbersome but fortunately there is a builtin command in Sage for Taylor expansions 

In [34]:
F.taylor(x,0,10)

1/47900160*x^10 - 1/1209600*x^8 + 1/30240*x^6 - 1/720*x^4 + 1/12*x^2 - 1/2*x + 1

In [56]:
type(F.taylor(x,0,10))

<class 'sage.symbolic.expression.Expression'>

We can convert this to a polynomial over QQ:
    

In [61]:
p = F.taylor(x,0,10).polynomial(QQ)
type(p)

<class 'sage.rings.polynomial.polynomial_rational_flint.Polynomial_rational_flint'>

For a polynomial we cna add a big-Oh

In [62]:
p.add_bigoh(12)

1 - 1/2*x + 1/12*x^2 - 1/720*x^4 + 1/30240*x^6 - 1/1209600*x^8 + 1/47900160*x^10 + O(x^12)

We can get coefficients of certain terms in Taylor expansions

In [35]:
F.taylor(x,0,10).coefficient(x^4)

-1/720

In [ ]:
# We can now write a function that returns the j-th Bernoulli number

In [38]:
def B(j):
    F = x / (e^x -1)
    return F.taylor(x,0,j).coefficient(x^j)*factorial(j)
    

[B(j) for  j in range(1,10)]


[-1/2, 1/6, 0, -1/30, 0, 1/42, 0, -1/30, 0]

## Variable scopes:
In general variables inside functions are **local** so they don't affect other variables "outside":

In [52]:
x=1
def my_function(x):
    print(f"x inside function is {x}")
    x = 3
    print(f"x inside function is {x}")
    
my_function(2)
print(x)

x inside function is 2
x inside function is 3
1


However, the symbolic variable `x` is special and it gets it's scope from the outside. 

In [54]:
def B(j):
    F = x / (e^x -1)  # x = 1 so F is a constant
    return F.taylor(x,0,j).coefficient(x^j)*factorial(j)
[B(j) for  j in range(1,10)]

[0, 0, 0, 0, 0, 0, 0, 0, 0]

In [55]:
def B(j):
    var('x')
    F = x / (e^x -1)
    y=2
    return F.taylor(x,0,j).coefficient(x^j)*factorial(j)
    

[B(j) for  j in range(1,10)]


[-1/2, 1/6, 0, -1/30, 0, 1/42, 0, -1/30, 0]

## Bernoulli polynomials
The Bernoulli polynomials can also be given by a generating series
$$
 F(x,t) = \frac{te^{xt}}{e^{t}-1} = \sum_{n=0}^{\infty} B_n(x)\frac{t^n}{n!}
$$

**Exercise**: Write a function that computes the $n$-th Bernoulli polynomial, $B_n(t)$ using the generating function. 